In [37]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from sklearn.metrics import silhouette_score
from tqdm import tqdm

In [2]:
df = pd.read_parquet('/Users/manu/Desktop/SUP/Projet 2/AI_model_urban_mobility/data/df_final_15min_NoNan_20250505.parquet')

# Gestion des données temporelles

Afin de capturer les profils temporels dans le modèle, nous appliquons un clustering basé sur le jour de la semaine, le créneau horaire, les vacances et les jours fériés. Ce cluster reflète des schémas typiques de trafic et est ensuite utilisé comme variable d’entrée dans le modèle prédictif.

## Calcul des moyennes horaires par tronçon

Création d'une variable quarter_hour qui identifie chaque créneau de 15 minutes dans la journée, afin de capturer plus finement la variation temporelle du trafic dans le modèle.

In [ ]:
df['quarter_hour'] = df['hour'] * 4 + df['minute'] // 15

## Touver le nombre de cluster ideal

In [38]:
def find_best_k(df_t, max_k=10):
    X = df_t[['weekday', 'quarter_hour', 'is_vacances', 'is_ferie']].copy()
    X_scaled = StandardScaler().fit_transform(X)

    best_score = -1
    best_k = 2

    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)

        if score > best_score:
            best_score = score
            best_k = k

    return best_k

In [39]:
# Application sur un echantillon de 200 tronçons aléatoire
troncons_sample = np.random.choice(df['troncon_enc'].unique(), size=200, replace=False)
k_list = []

for troncon in tqdm(troncons_sample, desc="Recherche de k optimal"):
    df_t = df[df['troncon_enc'] == troncon]
    if len(df_t) >= 10: 
        k_opt = find_best_k(df_t, max_k=10)
        k_list.append(k_opt)

# Trouver la valeur la plus fréquente
mode_k = Counter(k_list).most_common(1)[0][0]
print(f"Nombre optimal de clusters trouvé sur 200 tronçons : {mode_k}")

Recherche de k optimal: 100%|██████████| 200/200 [03:39<00:00,  1.10s/it]

Nombre optimal de clusters trouvé sur 200 tronçons : 8


## Aggrégation

Réduit la granularité temporelle et avoir des profils plus stables et moins bruités, ce qui facilite le clustering et donne des clusters plus représentatifs.

In [41]:
agg = df.groupby(['troncon_enc', 'weekday', 'quarter_hour', 'is_vacances', 'is_ferie']).agg({
    'code_couleur': 'mean'
}).reset_index()


## Application du clustering

In [43]:
def compute_final_time_clusters(df_agg, n_clusters=5):
    all_clusters = []

    for troncon in tqdm(df_agg['troncon_enc'].unique(), desc="Clustering final par tronçon"):
        df_t = df_agg[df_agg['troncon_enc'] == troncon].copy()
        X = df_t[['weekday', 'quarter_hour', 'is_vacances', 'is_ferie']]
        X_scaled = StandardScaler().fit_transform(X)

        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
        df_t['time_cluster'] = kmeans.fit_predict(X_scaled)

        all_clusters.append(df_t)

    return pd.concat(all_clusters, ignore_index=True)


In [44]:
# Application sur les données agrégées
df_clusters = compute_final_time_clusters(agg, n_clusters=5)

Clustering final par tronçon: 100%|██████████| 749/749 [00:10<00:00, 74.71it/s]


In [45]:
# Merge sur les colonnes clés temporelles et tronçon
df_final = df.merge(
    df_clusters[['troncon_enc', 'weekday', 'quarter_hour', 'is_vacances', 'is_ferie', 'time_cluster']],
    on=['troncon_enc', 'weekday', 'quarter_hour', 'is_vacances', 'is_ferie'],
    how='left'
)